# Ayudantía 7: Vizualisación interactiva 

Lo que veremos esta ayudantía será un repaso de `ipywidgets` y como usarlo en conjunto con altair con tal de crear vizualizaciones interesantes de datos. Para ello repasaremos la base de datos de casos de Covid-19 en chile en Valparaiso, y veremos como podemos hacerla interactiva

In [1]:
import ipywidgets as widgets 
import numpy as np
import pandas as pd
import altair as alt

alt.data_transformers.disable_max_rows()
%matplotlib inline

Ahora, invoquiemos los datos entregados por el ministerio de ciencias, estos se pueden encontrar en el siguiente [repo de github](https://github.com/MinCiencia/Datos-COVID19). Tambien le daremos un mejor formato al nombre de las columnas, y vamos a reiniciar el indice, ya que por defecto el nombre de la region queda como indice

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto1/Covid-19.csv")
df.columns = df.columns.str.lower().str.replace(' ', '_').str.strip()
df.head()

,region,codigo_region,comuna,codigo_comuna,poblacion,2020-03-30,2020-04-01,2020-04-03,2020-04-06,2020-04-08,...,2020-10-05,2020-10-09,2020-10-12,2020-10-16,2020-10-19,2020-10-24,2020-10-26,2020-10-30,2020-11-02,tasa
0,Arica y Parinacota,15,Arica,15101.0,247552.0,6.0,6.0,12.0,41.0,63.0,...,8772.0,8996.0,9138.0,9343.0,9472.0,9679.0,9763.0,9888.0,9974.0,4029.1
1,Arica y Parinacota,15,Camarones,15102.0,1233.0,0.0,0.0,0.0,0.0,0.0,...,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,2270.9
2,Arica y Parinacota,15,General Lagos,15202.0,810.0,0.0,0.0,0.0,0.0,0.0,...,64.0,64.0,64.0,64.0,64.0,64.0,64.0,64.0,64.0,7901.2
3,Arica y Parinacota,15,Putre,15201.0,2515.0,0.0,0.0,0.0,0.0,0.0,...,70.0,70.0,70.0,70.0,70.0,71.0,72.0,72.0,72.0,2862.8
4,Arica y Parinacota,15,Desconocido Arica y Parinacota,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,36.0,36.0,40.0,43.0,43.0,43.0,44.0,36.0,37.0,NaN


Ahora, lo que quiero hacer es graficar todos los casos de todas las regiones por separado, para eso hay que sacar las las columnas que estan de más, y juntar los datos. 

In [3]:
df.drop(columns = ['codigo_region','comuna', 'codigo_comuna', 'tasa'], inplace = True)
df.head()

,region,poblacion,2020-03-30,2020-04-01,2020-04-03,2020-04-06,2020-04-08,2020-04-10,2020-04-13,2020-04-15,...,2020-10-02,2020-10-05,2020-10-09,2020-10-12,2020-10-16,2020-10-19,2020-10-24,2020-10-26,2020-10-30,2020-11-02
0,Arica y Parinacota,247552.0,6.0,6.0,12.0,41.0,63.0,87.0,115.0,124.0,...,8597.0,8772.0,8996.0,9138.0,9343.0,9472.0,9679.0,9763.0,9888.0,9974.0
1,Arica y Parinacota,1233.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,27.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0
2,Arica y Parinacota,810.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,64.0,64.0,64.0,64.0,64.0,64.0,64.0,64.0,64.0,64.0
3,Arica y Parinacota,2515.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,67.0,70.0,70.0,70.0,70.0,70.0,71.0,72.0,72.0,72.0
4,Arica y Parinacota,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37.0,36.0,36.0,40.0,43.0,43.0,43.0,44.0,36.0,37.0


Y ahora, con un `groupby` podemos obtener los datos totales por region

In [4]:
tot_reg = df.groupby('region').apply(lambda x: x.drop(columns = ['region']).sum()).reset_index()
tot_reg

,region,poblacion,2020-03-30,2020-04-01,2020-04-03,2020-04-06,2020-04-08,2020-04-10,2020-04-13,2020-04-15,...,2020-10-02,2020-10-05,2020-10-09,2020-10-12,2020-10-16,2020-10-19,2020-10-24,2020-10-26,2020-10-30,2020-11-02
0,Antofagasta,691854.0,29.0,32.0,49.0,65.0,72.0,105.0,149.0,176.0,...,21055.0,21236.0,21415.0,21599.0,21760.0,21915.0,22153.0,22245.0,22426.0,22542.0
1,Araucanía,1014343.0,187.0,292.0,414.0,541.0,628.0,680.0,758.0,823.0,...,7867.0,8334.0,9005.0,9455.0,10056.0,10572.0,11457.0,11839.0,12540.0,13101.0
2,Arica y Parinacota,252110.0,6.0,6.0,12.0,41.0,63.0,87.0,115.0,124.0,...,8792.0,8970.0,9194.0,9340.0,9548.0,9677.0,9885.0,9971.0,10088.0,10175.0
3,Atacama,314709.0,0.0,0.0,4.0,4.0,5.0,12.0,13.0,13.0,...,7170.0,7252.0,7390.0,7507.0,7593.0,7700.0,7804.0,7857.0,7941.0,7985.0
4,Aysén,107297.0,0.0,0.0,4.0,5.0,4.0,4.0,7.0,7.0,...,499.0,569.0,640.0,741.0,835.0,908.0,1034.0,1052.0,1105.0,1146.0
5,Biobío,1663696.0,72.0,224.0,263.0,360.0,409.0,439.0,507.0,542.0,...,28033.0,28606.0,29294.0,30057.0,31005.0,31746.0,32936.0,33385.0,34174.0,34884.0
6,Coquimbo,836096.0,21.0,23.0,31.0,49.0,54.0,59.0,66.0,66.0,...,13897.0,14041.0,14184.0,14264.0,14394.0,14463.0,14591.0,14626.0,14710.0,14753.0
7,Los Lagos,891440.0,104.0,169.0,198.0,250.0,283.0,304.0,380.0,390.0,...,10259.0,10795.0,11382.0,11942.0,12663.0,13288.0,14333.0,14774.0,15544.0,16293.0
8,Los Ríos,405835.0,12.0,54.0,57.0,98.0,108.0,118.0,137.0,150.0,...,2131.0,2263.0,2472.0,2622.0,2780.0,2958.0,3211.0,3337.0,3594.0,3900.0
9,Magallanes,178362.0,34.0,97.0,154.0,218.0,223.0,318.0,415.0,444.0,...,9281.0,9935.0,10844.0,11438.0,11856.0,12181.0,12666.0,12876.0,13177.0,13498.0


Y, lo ultimo que nos queda por hacer es un melt, de forma que las fechas aparezcan en una columnas. Tambien aprovecharemos de convertir la columna fecha en a *DateTime*

In [5]:
tot_reg_melt = tot_reg.melt(id_vars = ['region', 'poblacion'], var_name="fecha", value_name='casos')
tot_reg_melt.fecha = pd.to_datetime(tot_reg_melt.fecha)
tot_reg_melt.head()

,region,poblacion,fecha,casos
0,Antofagasta,691854.0,2020-03-30,29.0
1,Araucanía,1014343.0,2020-03-30,187.0
2,Arica y Parinacota,252110.0,2020-03-30,6.0
3,Atacama,314709.0,2020-03-30,0.0
4,Aysén,107297.0,2020-03-30,0.0


Ahora vamor a graficar:

In [6]:
alt.Chart(tot_reg_melt).mark_line().encode(
    x = 'fecha:T',
    y = 'casos:Q',
    color = 'region',
    tooltip = ['fecha', 'casos', 'region']
).interactive()

alt.Chart(...)

Un poco de forma esperada Santiago eclipsa al resto de las regiones, para solucionar esto lo que haremos será convertir los datos a una tasa cada 100.000 habitantes (asumiremos que la poblacion no cambia en el tiempo por simplcidad)

In [7]:
tot_reg_melt['tasa_casos'] = tot_reg_melt.casos/tot_reg_melt.poblacion*100000
tot_reg_melt.head()

,region,poblacion,fecha,casos,tasa_casos
0,Antofagasta,691854.0,2020-03-30,29.0,4.191636
1,Araucanía,1014343.0,2020-03-30,187.0,18.435578
2,Arica y Parinacota,252110.0,2020-03-30,6.0,2.379914
3,Atacama,314709.0,2020-03-30,0.0,0.000000
4,Aysén,107297.0,2020-03-30,0.0,0.000000


In [8]:
alt.Chart(tot_reg_melt).mark_line().encode(
    x = 'fecha:T',
    y = 'tasa_casos:Q',
    color = 'region',
    tooltip = ['fecha', 'tasa_casos', 'region']
).interactive()

alt.Chart(...)

Ahora, ¿cómo lo hacemos intereactivo?. El como es relativamente fácil, `IpyWidgets`. El qué es algo más complicado, ¿Qué queremos hacer con el gráfico? En este caso yo haré tres cosas, una caja que nos pregunté cual de los dos gráfico queremos, un selector de regiones, y un selector de rango de fechas. Pero antes un pequeño repaso de como funciona la libreria. En resumen, nosotros creamos un widget, este nos entrega una valor dependiendo de cual sea lo que selecionamos, y usamos la función interact para hacerlo interactuar con una funcion de nuestra creacion.

Cremos un widget para ver como funcionan

In [9]:
slider = widgets.IntSlider()
slider

IntSlider(value=0)

Podemos ver que `IntSlider` nos entrega una barrita en la que podemos seleccionar los valores. Ahora usemos definamos una función simple y usemos `interact`

In [10]:
def f(x):
    print(x)

In [11]:
widgets.interact(f, x=slider)

interactive(children=(IntSlider(value=0, description='x'), Output()), _dom_classes=('widget-interact',))

<function __main__.f(x)>

Podemos ver que el valor que esta escupiendo la función es el que designamos en la barrita, notemos tambien que tuvimos que decirle que valor usar, en este caso `x`, estor será importante más adelante. Puede encontrar una lista de todos los *widdgets* que existen en la [documentacion](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html). Mientras hacemos el ejemplo podremos ver algunos extra

Empezemos el ejemplo, lo primero que haremos será crear el selecior de si el gráfico es el de tasa o el de casos totales, para esto usaremos [`RadioButtons`](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html#RadioButtons), primero crearemos el widget:

In [12]:
tipo_de_grap = widgets.RadioButtons(options=['casos', 'tasa_casos'], #opciones de los botones
                                    description='Col. graf.:', #descripcion del boton
                                   )
tipo_de_grap

RadioButtons(description='Col. graf.:', options=('casos', 'tasa_casos'), value='casos')

Ya tenemos nuestro widget, ahora definamos una funcion que nos entrege el grafico que tenemos con opciones para lo que estamos buscando modoficar.

In [13]:
def graf_casos(df, col):
    viz = alt.Chart(df).mark_line().encode(
              x = 'fecha:T',
              y = f'{col}:Q',
              color = 'region',
              tooltip = ['fecha', 'tasa_casos', 'region']
          ).interactive()
    return viz

Y ahora, usaremos interact para generar al grafico:

In [14]:
widgets.interact(graf_casos, df = widgets.fixed(tot_reg_melt), col = tipo_de_grap)

interactive(children=(RadioButtons(description='Col. graf.:', options=('casos', 'tasa_casos'), value='casos'),…

<function __main__.graf_casos(df, col)>

Genial! Notemos el uso de `fixed()` este es un *widgtet* que en verdad no hace nada, solo fija el valor de una variable, pero es necesario al usar `interact`, ya que este nos exige que le entregemos *widgets* para todas las variables.

Ahora haremos el selector de regiones, para esto usaremos [`Dropdown`](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html#Dropdown), la primera opción será hacerlas todas, y luego region por region

In [15]:
regiones = list(tot_reg_melt.region.unique())
select_reg = widgets.Dropdown(
                              options = ['Todas'] + regiones, #valores paara elegir
                              value = 'Todas', #valor por defecto
                              description='Region:', #descripcion
                             )
select_reg

Dropdown(description='Region:', options=('Todas', 'Antofagasta', 'Araucanía', 'Arica y Parinacota', 'Atacama',…

Ahora modifiquemos nuestra funcion:

In [16]:
def graf_casos(df, col, reg):
    if reg != 'Todas':
        df = df[df.region == reg]
    viz = alt.Chart(df).mark_line().encode(
              x = 'fecha:T',
              y = f'{col}:Q',
              color = 'region',
              tooltip = ['fecha', 'tasa_casos', 'region']
          ).interactive()
    return viz

In [17]:
widgets.interact(graf_casos, df = widgets.fixed(tot_reg_melt), col = tipo_de_grap, reg = select_reg)

interactive(children=(RadioButtons(description='Col. graf.:', options=('casos', 'tasa_casos'), value='casos'),…

<function __main__.graf_casos(df, col, reg)>

Ahora solo nos queda una, y es la más difil, crear un slider para eleguir las fechas. En verdad no es tan complejo, usaremos [`SelectionRangeSlider`](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html#SelectionRangeSlider), pero este nos pide más cosas para poder usarlo:

In [18]:
start_date = tot_reg_melt.fecha.min() #primera fecha
end_date = tot_reg_melt.fecha.max() #ultima fecha

dates = tot_reg_melt.fecha.unique() #lista de fechas en el dataframe

options = [(pd.to_datetime(str(date)).strftime(' %d %b %Y '), pd.to_datetime(str(date)) ) for date in dates] #esto genera los valores de la forma (02 Nov 2020, fecha real)
index = (0, len(options)-1) #los indices que usiaremos 

rango_fechas = widgets.SelectionRangeSlider(
    options=options,
    index=index,
    description='Fechas',
    orientation='horizontal',
    layout={'width': '500px'} #largo del widget
)

In [19]:
rango_fechas

SelectionRangeSlider(description='Fechas', index=(0, 64), layout=Layout(width='500px'), options=((' 30 Mar 202…

Por ultimo nos queda modificar la funcion de forma que filtre los datos antes de graficar:

In [20]:
def graf_casos(df, col, reg, date_range):
    mask = (df.fecha > date_range[0]) & (df.fecha <= date_range[1])
    df = df.loc[mask]
    if reg != 'Todas':
        df = df[df.region == reg]
    viz = alt.Chart(df).mark_line().encode(
              x = 'fecha:T',
              y = f'{col}:Q',
              color = 'region',
              tooltip = ['fecha', 'tasa_casos', 'region']
          ).interactive()
    return viz

Ahora, con esto listo solo hay que invcar al `interact`

In [21]:
widgets.interact(graf_casos, df = widgets.fixed(tot_reg_melt), col = tipo_de_grap, reg = select_reg, date_range = rango_fechas)

interactive(children=(RadioButtons(description='Col. graf.:', options=('casos', 'tasa_casos'), value='casos'),…

<function __main__.graf_casos(df, col, reg, date_range)>

Con esto ya deberían tenre una idea, ahora les toca a ustedes implementar!

## Ejercicios
### Ejercicio 1

Para este ejercio usaremos el *dataset* de *Kaggle* [*Amazon Top 50 Bestselling Books 2009 - 2019*](https://www.kaggle.com/sootersaalu/amazon-top-50-bestselling-books-2009-2019), que, citando de ahí mismo:

*Dataset on Amazon's Top 50 bestselling books from 2009 to 2019. Contains 550 books, data has been categorized into fiction and non-fiction using Goodreads*

Invoquemos estos datos y hagamos el típico formato de columnas

In [22]:
books = pd.read_csv('data/bestsellers_with_categories.csv')
books.columns = books.columns.str.lower().str.replace(' ','_')
books.head()

,name,author,user_rating,reviews,price,year,genre
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8,2016,Non Fiction
1,11/22/63: A Novel,Stephen King,4.6,2052,22,2011,Fiction
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15,2018,Non Fiction
3,1984 (Signet Classics),George Orwell,4.7,21424,6,2017,Fiction
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12,2019,Non Fiction


Comenzaremos por lo básico, gráfica (para lo que vamos a usar recomiendo usar Altair), de forma que en el eje x esté `user_rating`, en el eje y esté `reviews`, el color sea 'genre' y un *tooltop* que nos muestre estos dos datos junto con `'name'` y` 'author'`.

In [23]:
alt.Chart(books).mark_circle(size=60).encode(
    x='user_rating:Q',
    y='reviews:Q',
    color='genre',
    tooltip=['name', 'author','user_rating','reviews']
).interactive()

alt.Chart(...)

Ahora, tenemos más columnas con datos, construiremos un *Widget* que nos permita elegir que columnas queremos mostrar al graficar, para esto construye dos `RadioButtons` que tengan por opciones las columnas de valores numericos de los datos y descripción el eje que modifican.

In [24]:
w_eje_x = widgets.RadioButtons(
                               options=['user_rating', 'reviews', 'price', 'year'],
                               description='Eje x:',
                              )

In [25]:
w_eje_y = widgets.RadioButtons(
                               options=['user_rating', 'reviews', 'price', 'year'],
                               description='Eje y:',
                              )

Ahora, contruye la función que nos entrege al gráfico que queremos, donde los datos mostrados sean los ejes selacionados con los *widgets*

In [26]:
def book_alt(df, eje_x, eje_y):
    return (alt.Chart(books)
               .mark_circle(size=60)
               .encode(
                       x= eje_x,
                       y= eje_y,
                       color='genre',
                       tooltip=['name', 'author', eje_x, eje_y]
                      )
               .interactive()
           )

In [27]:
widgets.interact(book_alt, df = widgets.fixed(books), eje_x = w_eje_x, eje_y = w_eje_y )

interactive(children=(RadioButtons(description='Eje x:', options=('user_rating', 'reviews', 'price', 'year'), …

<function __main__.book_alt(df, eje_x, eje_y)>

¡Genial! Un solo problema, al seleccionar `year` nuestro gráfico se rompe, esto es porque trata esa variable como una cuiantitativa en vez de una nominal, modifica la función de forma qué, si se selecciona `year` en una de variables, el tipo de dato que muestra sea nominal.

In [28]:
def book_alt(df, eje_x, eje_y):
    dtype_x, dtype_y = 'Q','Q'
    if eje_x == 'year':
        dtype_x = 'N'
    if eje_y == 'year':
        dtype_y = 'N'
    
    return (alt.Chart(books)
               .mark_circle(size=60)
               .encode(
                       x= f'{eje_x}:{dtype_x}',
                       y= f'{eje_y}:{dtype_y}',
                       color='genre',
                       tooltip=['name', 'author',eje_x,eje_y]
                      )
               .interactive()
           )

In [29]:
widgets.interact(book_alt, df = widgets.fixed(books), eje_x = w_eje_x, eje_y = w_eje_y )

interactive(children=(RadioButtons(description='Eje x:', options=('user_rating', 'reviews', 'price', 'year'), …

<function __main__.book_alt(df, eje_x, eje_y)>

Aun hay más cosas que podriamos agregar, pero ¿y si quremos filtrar los datos que vamos a mostrar? Esto es lo que veremos en la siguiente actividad

## Ejercicios
### Ejercicio 2

Para este ejercicio usaremos el dataset de Kaggle ["COVID-19's Impact on Airport Traffic"](https://www.kaggle.com/terenceshin/covid19s-impact-on-airport-traffic), citando desde ahí:

*This dataset shows traffic to and from the Airport as a Percentage of the Traffic volume during the baseline period. The baseline period used for computing this metric is from 1st Feb to 15th March 2020. The dataset gets updated monthly.*

Para eso lo vamos a llamar, y luego haremos lo típico, arreglar el formato de las columnas y sacar las columnas que no usaremos; también converstiremos la columnas `Date` al tipo de dato que corresponde

In [30]:
air = pd.read_csv('data/covid_impact_on_airport_traffic.csv')
air.drop(columns = ['AggregationMethod', 'Version','ISO_3166_2','Centroid', 'Geography'], inplace = True)
air.columns = air.columns.str.strip().str.lower() 
air.date = pd.to_datetime(air.date)
air.head()

,date,airportname,percentofbaseline,city,state,country
0,2020-07-05,Kingsford Smith,52,Sydney,New South Wales,Australia
1,2020-05-28,Kingsford Smith,61,Sydney,New South Wales,Australia
2,2020-05-07,Kingsford Smith,62,Sydney,New South Wales,Australia
3,2020-06-24,Kingsford Smith,58,Sydney,New South Wales,Australia
4,2020-08-05,Kingsford Smith,20,Sydney,New South Wales,Australia


Ahora, realiza un plot de los datos (para lo que vamos a usar recomiendo usar Altair), con las fechas en el eje X, `mean(percentofbaseline)` (esto lo hacemos porque hay mas de una entrada por pais por fecha) en el eje Y, y los paises separados por colores: 

In [31]:
alt.Chart(air).mark_line().encode(
    x='date:T',
    y='mean(percentofbaseline):Q',
    color = 'country'
    )

alt.Chart(...)

Podemos ver que tenemos se ven fuertemente afectados por un vaiven semanal, esto es malo porque no nos permite apreciar ver las tendencias que se puedan formar en los datos, para esto quedaria mejor porder ver los datos semana a semana, ya que no es el objetivo de la actividad de esta semana les entrego esta pega hecha, pero de todas formas quiero que describan paso a paso que es lo que está haciendo el codigo de abajo:

In [32]:
air = (air.set_index('date')
          .groupby(['country'
                    , 'state'
                    , 'city'
                    , 'airportname']
                  )
          .apply(lambda x: x.resample('W', label='right').mean())
          .reset_index()
      )

**¿Que está haciendo la celda de arriba?**

**Respuesta:**

Ahora, veamos como queda nuestro nuevo grafico luego de lo que hicimos:

In [33]:
alt.Chart(air).mark_line().encode(
    x='date:T',
    y='mean(percentofbaseline):Q',
    color = 'country'
    )

alt.Chart(...)

Ahora, queremos hacer que esta vizualización sea interactiva. Lo primero que nos permita seleccionar entre que fechas queremos ver los datos, para eso utiliza `SelectionRangeSlider` para crear un *slider* que vaya desde la primera fecha de los datos, hazta la ultima, en incrementos de una semana

Hint: para la lista de todos los valores te recomiento usar ``

In [34]:
start_date = air.date.min()
end_date = air.date.max()

dates = pd.date_range(start_date, end_date, freq='W') #esto vendra entregado

options = [(date.strftime(' %d %b %Y '), date) for date in dates] #esto vendra entregado
index = (0, len(options)-1)

selection_range_slider = widgets.SelectionRangeSlider(
    options=options,
    index=index,
    description='Fechas',
    orientation='horizontal',
    layout={'width': '500px'}
)

In [35]:
selection_range_slider

SelectionRangeSlider(description='Fechas', index=(0, 30), layout=Layout(width='500px'), options=((' 22 Mar 202…

Con nuestro *slider* funcionando, ahora crea la funcion que nos entregará el gráfico entre las fechas pedidas. Para esto, te recomiendo filtrar el *DataFrame* y luego generar la vizualización:

In [36]:
def altair_plot(df, date_range):
    mask = (df['date'] > date_range[0]) & (df['date'] <= date_range[1])
    df = df.loc[mask]
    viz = alt.Chart(df).mark_line().encode(
    x='date:T',
    y='mean(percentofbaseline):Q',
    color = 'country'
    )
    return viz

In [37]:
widgets.interact(altair_plot, date_range=selection_range_slider, df=widgets.fixed(air));

interactive(children=(SelectionRangeSlider(description='Fechas', index=(0, 30), layout=Layout(width='500px'), …

Supongamos que queremos seleccionar que paises podemos ver, para esto usaremos 4 `Checkbox` con los valores para cada pais. 

Hint: Puedes usar el metodo `[i for i in countries]` si tienes una lsita de los paises

In [38]:
check_boxes = [widgets.Checkbox(
                                value=False,
                                description=i,
                                disabled=False,
                                indent=False
                               ) for i in air.country.unique()
              ]

Ahora, haz algo similar a lo anterior, recuerda que cada caja entrega un valor *booleano*, así que filtra los datos antes de meterlos a la vizualización

In [39]:
def altair_plot(df, date_range, show_aus , show_cnd , show_cl , show_usa ):
    if not show_aus:
        mask = df.country !=  'Australia'
        df = df.loc[mask]
    if not show_cnd:
        mask = df.country !=  'Canada'
        df = df.loc[mask]
       
    if not show_cl:
        mask = df.country !=  'Chile'
        df = df.loc[mask]
    
    if not show_usa:
        mask = df.country !=  'United States of America (the)'
        df = df.loc[mask]
        
    mask = (df.date > date_range[0]) & (df.date <= date_range[1])
    df = df.loc[mask]
    viz = alt.Chart(df).mark_line().encode(
                                           x='date:T',
                                           y='mean(percentofbaseline):Q',
                                           color = 'country', 
                                          )
    return viz

In [40]:
widgets.interact(altair_plot
                 , date_range = selection_range_slider
                 , df = widgets.fixed(air)
                 , show_aus =  check_boxes[0]
                 , show_cnd = check_boxes[1]
                 , show_cl = check_boxes[2]
                 , show_usa = check_boxes[3]
                )

interactive(children=(SelectionRangeSlider(description='Fechas', index=(0, 30), layout=Layout(width='500px'), …

<function __main__.altair_plot(df, date_range, show_aus, show_cnd, show_cl, show_usa)>

Con esto ya te puedes hacer una idea de como podemos usar *ipywidgets* para la vizualizacion de datos .